In [ ]:
import torch
from torch import Tensor
import torch.nn as nn
from torch.nn import functional as F
import random
from typing import Dict
import pandas as pd
from collections import defaultdict, deque, OrderedDict
import numpy as np

# checkpoint = torch.load('/data/dataset/Epickitchens100_clips/epic_checkpoint-2400.pth')
v2_checkpoint = torch.load('/data/joohyun7u/project/CAST/clip_models/videomae_b_1600_checkpoint.pth', map_location=torch.device('cpu'))

/tmp/ipykernel_1282216/3761979770.py:12: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  v2_checkpoint = torch.load('/data/joohyun7u/project/CAST/clip_models/videomae_b_1600_c

In [ ]:
for key in checkpoint['model'].keys():
    print(key)

In [ ]:
for key in v2_checkpoint['model'].keys():
    print(key)

In [ ]:
import pickle
import pandas as pd

# 파일 경로
hd_anno_path = '/data/joohyun7u/project/CAST/hd-epic-annotations/narrations-and-action-segments/HD_EPIC_Narrations.pkl'


# pickle 파일을 바이너리 읽기 모드(rb)로 엽니다.
with open(hd_anno_path, 'rb') as f:
    # 파일 내용을 로드합니다.
    data = pickle.load(f)


In [ ]:
import pickle
import pandas as pd

hd_anno_path = '/data/joohyun7u/project/CAST/hd-epic-annotations/narrations-and-action-segments/HD_EPIC_Narrations.json'

hd_data = pd.read_json(hd_anno_path, lines=True)


In [ ]:
hd_data[hd_data['unique_narration_id'] == 'P02-20240211-141313-664']

,unique_narration_id,participant_id,video_id,narration,start_timestamp,end_timestamp,nouns,verbs,pairs,main_actions,verb_classes,noun_classes,pair_classes,main_action_classes,hands,narration_timestamp
13967,P02-20240211-141313-664,P02,P02-20240211-141313,place the recipe book down onto the countertop...,1423.45,1423.77,"[recipe book, countertop]",[place down],"[[place down, recipe book]]","[[place down, recipe book]]",[1],"[140, 42]","[[1, 140]]","[[1, 140]]",[right hand],1423.633573


In [ ]:
hd_data

,unique_narration_id,participant_id,video_id,narration,start_timestamp,end_timestamp,nouns,verbs,pairs,main_actions,verb_classes,noun_classes,pair_classes,main_action_classes,hands,narration_timestamp
0,P01-20240202-110250-1,P01,P01-20240202-110250,Open the upper cupboard by holding the handle...,7.440,8.75000,"[upper cupboard, handle of cupboard]","[open, hold]","[[open, upper cupboard], [hold, handle of cupb...","[[open, upper cupboard]]","[3, 34]","[3, 3]","[[3, 3], [34, 3]]","[[3, 3]]",[left hand],8.000000
1,P01-20240202-110250-2,P01,P01-20240202-110250,Stretch the left hand inside the cupboard in ...,8.850,9.36000,"[cupboard, lower shelf, mug, far side of cupbo...","[stretch, reach]","[[stretch, left hand], [reach, mug]]","[[stretch, left hand]]","[50, 98]","[3, 247, 13, 3]","[[50, 11], [98, 13]]","[[50, 11]]",[left hand],9.666667
2,P01-20240202-110250-3,P01,P01-20240202-110250,Pick up a mug from the lower shelf of the cup...,9.340,11.16000,"[mug, lower shelf of cupboard]",[pick up],"[[pick up, mug]]","[[pick up, mug]]",[0],"[13, 3]","[[0, 13]]","[[0, 13]]",[left hand],10.833333
3,P01-20240202-110250-4,P01,P01-20240202-110250,Turn the mug in the left hand to face the cam...,11.150,11.60000,"[mug, camera]","[turn, face]","[[turn, mug], [face, camera]]","[[turn, mug]]","[23, 11]","[13, 297]","[[23, 13], [11, 297]]","[[23, 13]]",[left hand],11.466667
4,P01-20240202-110250-5,P01,P01-20240202-110250,Close the cupboard using the right hand.,11.500,12.18000,[cupboard],[close],"[[close, cupboard]]","[[close, cupboard]]",[4],[3],"[[4, 3]]","[[4, 3]]",[right hand],11.566667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59449,P09-20240623-120359-373-4,P09,P09-20240623-120359,rinse and wash the frying pan under the runnin...,1248.871,1250.47771,"[running tap, frying pan]","[wash, rinse]","[[rinse, frying pan], [wash, frying pan]]","[[rinse, frying pan]]","[2, 2]","[0, 5]","[[2, 5], [2, 5]]","[[2, 5]]",[right hand],1250.221000
59450,P09-20240623-120359-373-5,P09,P09-20240623-120359,rotate the frying pan in the kitchen sink with...,1250.380,1251.38000,"[frying pan, kitchen sink]",[rotate],"[[rotate, frying pan]]","[[rotate, frying pan]]",[23],"[5, 63]","[[23, 5]]","[[23, 5]]",[right hand],1251.505000
59451,P09-20240623-120359-373-6,P09,P09-20240623-120359,swirl the frying pan under the running tap wat...,1251.260,1253.07771,"[areas of surface, running tap water, frying pan]","[swirl, rinse]","[[swirl, frying pan], [rinse, areas of surface]]","[[swirl, frying pan]]","[15, 2]","[42, 27, 5]","[[15, 5], [2, 42]]","[[15, 5]]",[],1252.485000
59452,P09-20240623-120359-373-7,P09,P09-20240623-120359,tip the frying pan in the right hand to pour o...,1252.964,1253.96400,"[frying pan, water]","[tip, pour out]","[[tip, frying pan], [pour out, water]]","[[tip, frying pan]]","[1, 9]","[5, 27]","[[1, 5], [9, 27]]","[[1, 5]]",[right hand],1253.689000


In [ ]:
import pandas as pd

# CSV 파일 경로
train_csv_path = '/data/joohyun7u/project/CAST/dataset/epic/epic100_train_gpt2_xl.csv'
val_csv_path = '/data/joohyun7u/project/CAST/dataset/epic/epic100_val_gpt2_xl.csv'

# CSV 파일 로드
train_df = pd.read_csv(train_csv_path)
val_df = pd.read_csv(val_csv_path)

# 'narration_id'에서 주방 환경 ID (P0*) 추출
train_kitchens = set(train_df['narration_id'].str.split('_').str[0])
val_kitchens = set(val_df['narration_id'].str.split('_').str[0])

# 각 데이터셋의 주방 환경 출력
print(f"Train 주방 환경 ({len(train_kitchens)}개):")
print(sorted(list(train_kitchens)))
print("-" * 30)

print(f"Validation 주방 환경 ({len(val_kitchens)}개):")
print(sorted(list(val_kitchens)))
print("-" * 30)

# Train 데이터셋에 없는 Validation 주방 환경 찾기
unseen_kitchens_in_val = val_kitchens - train_kitchens

print(f"Train에서 보지 못한 Validation 주방 환경 ({len(unseen_kitchens_in_val)}개):")
print(sorted(list(unseen_kitchens_in_val)))
print("-" * 30)

# Train에서 보지 못한 주방 환경만 val 데이터에서 필터링
val_df['kitchen_id'] = val_df['narration_id'].str.split('_').str[0]
unseen_val_df = val_df[val_df['kitchen_id'].isin(unseen_kitchens_in_val)].copy()

# 불필요한 'kitchen_id' 열 삭제
unseen_val_df = unseen_val_df.drop(columns=['kitchen_id'])


print("Train에 없는 주방 환경만 포함된 Validation 데이터프레임:")
unseen_val_df

Train 주방 환경 (32개):
['P01', 'P02', 'P03', 'P04', 'P05', 'P06', 'P07', 'P08', 'P09', 'P10', 'P11', 'P12', 'P13', 'P14', 'P15', 'P16', 'P17', 'P19', 'P20', 'P21', 'P22', 'P23', 'P24', 'P25', 'P26', 'P27', 'P28', 'P29', 'P30', 'P31', 'P35', 'P37']
------------------------------
Validation 주방 환경 (32개):
['P01', 'P02', 'P03', 'P04', 'P05', 'P06', 'P07', 'P08', 'P09', 'P10', 'P11', 'P12', 'P13', 'P14', 'P15', 'P16', 'P17', 'P18', 'P19', 'P20', 'P21', 'P22', 'P23', 'P24', 'P25', 'P26', 'P27', 'P28', 'P29', 'P30', 'P31', 'P32']
------------------------------
Train에서 보지 못한 Validation 주방 환경 (2개):
['P18', 'P32']
------------------------------
Train에 없는 주방 환경만 포함된 Validation 데이터프레임:


,narration_id,verb_class,noun_class,action_class,verb,noun,narration,all_nouns,all_noun_classes,lavila_narrator,start_timestamp,stop_timestamp,start_frame,stop_frame
4525,P18_01_0,0,7,7,take,bowl,take bowl,['bowl'],[7],"['#C C picks a bowl.', '#C C picks a bowl from...",00:00:03.64,00:00:05.49,218,329
4526,P18_01_1,3,3,903,open,cupboard,open cupboard,['cupboard'],[3],['#C C hangs the plate on the wall with his le...,00:00:05.31,00:00:06.78,318,406
4527,P18_01_10,9,27,2727,pour-in,water,pour water in coffee machine,"['water', 'machine:coffee']","[27, 50]","['#C C puts on the coffee maker', '#C C puts o...",00:00:29.48,00:00:37.26,1768,2235
4528,P18_01_11,5,50,1550,put-in,pot:coffee,put coffee pot in coffee machine,"['pot:coffee', 'machine:coffee']","[50, 50]","['#C C closes the coffee maker.', '#C C closes...",00:00:37.36,00:00:41.81,2241,2508
4529,P18_01_12,3,3,903,open,cupboard,open cupboard,['cupboard'],[3],"['#C C opens a cupboard with her left hand', '...",00:00:42.46,00:00:43.34,2547,2600
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9663,P32_10_5,44,22,13222,spray,degreaser,spray degreaser,['degreaser'],[22],"['#C C picks a kitchen towel', '#C C picks the...",00:01:14.36,00:01:17.04,4461,4622
9664,P32_10_6,2,46,646,wash,cooker,wash cooker,['cooker'],[46],['#C C wipes the countertop with a kitchen tow...,00:01:18.69,00:01:33.75,4721,5625
9665,P32_10_7,2,46,646,rinse,cooker,rinse cooker,['cooker'],[46],['#C C picks the cloth from the gas cooker wit...,00:01:39.52,00:01:59.64,5971,7178
9666,P32_10_8,2,17,617,rinse,cloth,rinse cloth,['cloth'],[17],"['#C C closes tap', '#C C closes the tap', '#C...",00:02:00.48,00:02:09.11,7228,7746


# EpicSounds Unseen

In [ ]:
import pandas as pd
# 예측 결과 파일 경로
pred_csv_path = '/data/joohyun7u/project/CAST/log_rebuttal/EKSound_ASTCLIP_split_time_encoding_v5_MixSpecCut_AddNoise_UF4/pred_result.csv'
# pred_csv_path = '/data/joohyun7u/project/CAST/log_rebuttal/7011437_CAVA_EPSound_reproduce_eval/pred_result.csv'

# 예측 결과 파일 로드
pred_df = pd.read_csv(pred_csv_path)

# 'Video_id'에서 주방 환경 ID (P**) 추출
pred_df['kitchen_id'] = pred_df['video_id'].str.split('_').str[0]

# Train에서 보지 못한 주방 환경(P18, P32)에 대한 예측만 필터링
# unseen_kitchens_in_val = {'P18', 'P32'}
unseen_pred_df = pred_df[pred_df['kitchen_id'].isin(unseen_kitchens_in_val)].copy()

# Top-1 정확도(acc1) 계산
# 'Pred_Label'과 'True_Label'이 같으면 True(1), 다르면 False(0)
# 이 값들의 평균을 구하면 정확도가 됩니다.
acc1 = (unseen_pred_df['pred'] == unseen_pred_df['label']).mean()

print(f"Unseen kitchens ({', '.join(sorted(list(unseen_kitchens_in_val)))}) Top-1 Accuracy: {acc1*100:.4f}%")

# 필터링된 데이터프레임 확인
unseen_pred_df

Unseen kitchens (P18, P32) Top-1 Accuracy: 62.3389%


,video_id,pred,label,conf,kitchen_id
277,P18_01_4,metal-only collision,metal-only collision,0.7559,P18
278,P18_01_20,plastic-only collision,metal / paper collision,0.3607,P18
279,P18_02_6,water,water,0.7542,P18
280,P18_03_8,rustle,rustle,0.8732,P18
281,P18_03_24,rustle,rustle,0.8180,P18
...,...,...,...,...,...
8030,P32_09_17,metal-only collision,metal-only collision,0.8328,P32
8031,P32_10_14,metal-only collision,metal-only collision,0.8758,P32
8032,P32_10_30,footstep,footstep,0.6908,P32
8033,P32_10_46,metal-only collision,metal-only collision,0.3992,P32


# EK100 Unseen (P18, P32)

In [ ]:
import pandas as pd
# 예측 결과 파일 경로
pred_csv_path = '/data/joohyun7u/project/CAST/log_rebuttal/7012125_CAST_eval/pred_result.csv'

# 예측 결과 파일 로드
pred_df = pd.read_csv(pred_csv_path)

# 'Video_id'에서 주방 환경 ID (P**) 추출
pred_df['kitchen_id'] = pred_df['video_id'].str.split('_').str[0]

# Train에서 보지 못한 주방 환경(P18, P32)에 대한 예측만 필터링
# unseen_kitchens_in_val = {'P18', 'P32'}
unseen_pred_df = pred_df[pred_df['kitchen_id'].isin(unseen_kitchens_in_val)].copy()

# Top-1 정확도(acc1) 계산
# 'Pred_Label'과 'True_Label'이 같으면 True(1), 다르면 False(0)
# 이 값들의 평균을 구하면 정확도가 됩니다.
acc1 = ((unseen_pred_df['verb'] == unseen_pred_df['label_verb']) & (unseen_pred_df['noun'] == unseen_pred_df['label_noun'])).mean()

print(f"Unseen kitchens ({', '.join(sorted(list(unseen_kitchens_in_val)))}) Top-1 Accuracy: {acc1*100:.4f}%")

# 필터링된 데이터프레임 확인
unseen_pred_df

Unseen kitchens (P18, P32) Top-1 Accuracy: 39.1549%


,video_id,verb,noun,label_verb,label_noun,conf_verb,conf_noun,action,kitchen_id
755,P18_01_13,take,rice,take,filter,0.8581,0.1106,take rice,P18
756,P18_01_19,take,spoon,open,coffee,0.5093,0.3743,take spoon,P18
757,P18_01_24,close,drawer,close,drawer,0.5591,0.4435,close drawer,P18
758,P18_01_3,close,cupboard,close,cupboard,0.8705,0.8452,close cupboard,P18
759,P18_01_35,take,fork,take,knife,0.8353,0.6113,take fork,P18
...,...,...,...,...,...,...,...,...,...
9663,P32_10_22,wash,maker:coffee,dry,hob,0.3560,0.1827,wash maker:coffee,P32
9664,P32_10_28,put,cupboard,put,liquid:washing,0.4861,0.1729,put cupboard,P32
9665,P32_10_33,wash,sink,squeeze,cloth,0.2969,0.4703,wash sink,P32
9666,P32_10_39,throw,napkin,throw,paper,0.8622,0.4070,throw napkin,P32


# Unseen EK100

In [19]:
import pandas as pd
cast_pred_df = pd.read_csv('/data/joohyun7u/project/CAST/log_rebuttal/7012125_CAST_eval/pred_result.csv')
cast_unseen = cast_pred_df[cast_pred_df['video_id'].str.contains('P18') | cast_pred_df['video_id'].str.contains('P32')]
print(f"Unseen Kitchens Action Accuracy: {((cast_unseen['verb'] == cast_unseen['label_verb']) & (cast_unseen['noun'] == cast_unseen['label_noun'])).mean()*100:.2f}%")


cava_pred_df = pd.read_csv('/data/joohyun7u/project/CAST/log_rebuttal/7061629_CAVA_EK100/pred_result.csv')
cava_unseen = cava_pred_df[cava_pred_df['video_id'].str.contains('P18') | cava_pred_df['video_id'].str.contains('P32')]
print(f"Unseen Kitchens Action Accuracy: {((cava_unseen['verb'] == cava_unseen['label_verb']) & (cava_unseen['noun'] == cava_unseen['label_noun'])).mean()*100:.2f}%")


ca2st_pred_df = pd.read_csv('/data/joohyun7u/project/CAST/log_rebuttal/EK100_ASTBsquare_share_split_time_encoding_v5/pred_result.csv')
ca2st_unseen = ca2st_pred_df[ca2st_pred_df['video_id'].str.contains('P18') | ca2st_pred_df['video_id'].str.contains('P32')]
print(f"Unseen Kitchens Action Accuracy: {((ca2st_unseen['verb'] == ca2st_unseen['label_verb']) & (ca2st_unseen['noun'] == ca2st_unseen['label_noun'])).mean()*100:.2f}%")

Unseen Kitchens Action Accuracy: 39.15%
Unseen Kitchens Action Accuracy: 35.12%
Unseen Kitchens Action Accuracy: 42.35%


In [6]:
import argparse
import subprocess
import os
import pandas as pd
from concurrent.futures import ProcessPoolExecutor, as_completed
import time
from pathlib import Path

anno_file = Path('/data/joohyun7u/project/CAST/dataset/hd_epic_sounds/HD_EPIC_Sounds.csv')
RESIZED_PATH = Path("/data/dataset/HD-EPIC/hd-epic-resized")
DEST_PATH = Path("/data/dataset/HD-EPIC/hd-epic-sounds-trimmed3")
anno_df = pd.read_csv(anno_file)
anno_df['mp4_name'] = anno_df['video_id'].astype(str) + '-' + anno_df['start_sample'].astype(str)
anno_df['mp4_name2'] = anno_df['video_id'].astype(str) + '-' + anno_df['start_sample'].astype(str) + '-' + anno_df['stop_sample'].astype(str)
anno_df
# DEST_PATH에 있는 파일 목록을 가져옵니다.
dest_files = set([f.stem for f in DEST_PATH.glob("*.mp4")])

# anno_df의 mp4_name 컬럼을 set으로 만듭니다.
anno_mp4_names = set(anno_df['mp4_name'])

# DEST_PATH에는 있는데 anno_df에는 없는 파일 찾기
not_in_anno = dest_files - anno_mp4_names

print(f"DEST_PATH에 있는데 anno_df에 없는 파일 개수: {len(not_in_anno)}")
if not_in_anno:
    print("예시:", list(not_in_anno)[:10])
else:
    print("모든 파일이 anno_df에 존재합니다.")
    
anno_df


DEST_PATH에 있는데 anno_df에 없는 파일 개수: 50966
예시: ['P03-20240217-131219-18375984-18388752', 'P04-20240414-065311-32590943-32720256', 'P06-20240510-121619-22348464-22430592', 'P09-20240621-093545-132389136-132403728', 'P01-20240204-142301-1804032-2058768', 'P07-20240530-102543-70040112-70060704', 'P02-20240210-100308-9879264-9892224', 'P03-20240216-223126-584016-623040', 'P08-20240614-184945-5552544-5569776', 'P09-20240624-160737-74614560-74688048']


,participant_id,video_id,start_timestamp,stop_timestamp,start_sample,stop_sample,class,class_id,mp4_name,mp4_name2
0,P01,P01-20240202-110250,00:00:00.476,00:00:02.520,22848,120960,rustle,4,P01-20240202-110250-22848,P01-20240202-110250-22848-120960
1,P01,P01-20240202-110250,00:00:03.767,00:00:07.888,180816,378624,footstep,7,P01-20240202-110250-180816,P01-20240202-110250-180816-378624
2,P01,P01-20240202-110250,00:00:06.462,00:00:06.682,310176,320736,footstep,7,P01-20240202-110250-310176,P01-20240202-110250-310176-320736
3,P01,P01-20240202-110250,00:00:07.765,00:00:08.667,372720,416016,open / close,3,P01-20240202-110250-372720,P01-20240202-110250-372720-416016
4,P01,P01-20240202-110250,00:00:09.567,00:00:10.342,459216,496416,ceramic-only collision,14,P01-20240202-110250-459216,P01-20240202-110250-459216-496416
...,...,...,...,...,...,...,...,...,...,...
50963,P09,P09-20240625-103827,00:02:33.443,00:02:34.685,7365264,7424880,footstep,7,P09-20240625-103827-7365264,P09-20240625-103827-7365264-7424880
50964,P09,P09-20240625-103827,00:02:34.685,00:02:35.042,7424880,7442016,metal / plastic collision,11,P09-20240625-103827-7424880,P09-20240625-103827-7424880-7442016
50965,P09,P09-20240625-103827,00:02:35.844,00:02:37.667,7480512,7568016,metal-only collision,0,P09-20240625-103827-7480512,P09-20240625-103827-7480512-7568016
50966,P09,P09-20240625-103827,00:02:38.382,00:02:38.739,7602336,7619472,open / close,3,P09-20240625-103827-7602336,P09-20240625-103827-7602336-7619472


In [4]:
from pathlib import Path

# 중복된 mp4_name을 가진 파일 목록 추출
dup_mp4_name2 = anno_df[anno_df.duplicated('mp4_name', keep=False)]
dup_mp4_names = dup_mp4_name2['mp4_name'].unique()

# 실제로 DEST_PATH에 존재하는 중복 파일만 필터링
dup_files = [DEST_PATH / f"{name}.mp4" for name in dup_mp4_names if (DEST_PATH / f"{name}.mp4").exists()]

print(f"중복 mp4_name 파일 개수: {len(dup_files)}")
for f in dup_files:
    try:
        f.unlink()
        print(f"삭제됨: {f}")
    except Exception as e:
        print(f"삭제 실패: {f} ({e})")

중복 mp4_name 파일 개수: 0


In [7]:
len(anno_df['mp4_name2'].unique())
# mp4_name2가 2개 이상인 경우를 찾으려면, 중복된 mp4_name2 값을 가진 행을 출력하면 됩니다.
dup_mp4_name2 = anno_df[anno_df.duplicated('mp4_name2', keep=False)]
dup_mp4_name2

# # video_id가 같고 stop_sample도 같은 경우 찾기
# dup_vid_stop = anno_df[anno_df.duplicated(['video_id', 'stop_sample'], keep=False)]
# dup_vid_stop


,participant_id,video_id,start_timestamp,stop_timestamp,start_sample,stop_sample,class,class_id,mp4_name,mp4_name2
30586,P06,P06-20240510-090328,00:26:15.381,00:33:36.461,75618288,96790128,sizzling / boiling,29,P06-20240510-090328-75618288,P06-20240510-090328-75618288-96790128
30587,P06,P06-20240510-090328,00:26:15.381,00:33:36.461,75618288,96790128,background,13,P06-20240510-090328-75618288,P06-20240510-090328-75618288-96790128
32790,P06,P06-20240510-121619,00:00:24.100,00:00:25.100,1156800,1204800,metal / marble collision,28,P06-20240510-121619-1156800,P06-20240510-121619-1156800-1204800
32791,P06,P06-20240510-121619,00:00:24.100,00:00:25.100,1156800,1204800,metal / marble collision,28,P06-20240510-121619-1156800,P06-20240510-121619-1156800-1204800


In [3]:
anno_df[anno_df['mp4_name'] == 'P01-20240204-145458-28292544']
anno_df[anno_df['mp4_name'] == 'P01-20240203-135502-23089776']
# anno_df[anno_df['mp4_name'] == 'P01-20240203-121517-10219440']
# anno_df[anno_df['mp4_name'] == 'P01-20240202-161354-3144384']
# anno_df[anno_df['mp4_name'] == 'P01-20240202-161948-0']
anno_df[anno_df['mp4_name'] == 'P01-20240203-132119-46784448']
anno_df[anno_df['mp4_name'] == 'P03-20240216-185832-76258800']

,participant_id,video_id,start_timestamp,stop_timestamp,start_sample,stop_sample,class,class_id,mp4_name
13712,P03,P03-20240216-185832,00:26:28.725,01:13:28.256,76258800,211596288,kettle / mixer / appliance,33,P03-20240216-185832-76258800


In [20]:
!ffmpeg -i /data/dataset/HD-EPIC/hd-epic-resized/P01-20240204-145458_resized.mp4


ffmpeg version 6.1.1 Copyright (c) 2000-2023 the FFmpeg developers
  built with gcc 11.2.0 (Anaconda gcc)
  configuration: --prefix=/croot/ffmpeg_1743153284778/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_ --cc=/croot/ffmpeg_1743153284778/_build_env/bin/x86_64-conda-linux-gnu-cc --ar=/croot/ffmpeg_1743153284778/_build_env/bin/x86_64-conda-linux-gnu-ar --nm=/croot/ffmpeg_1743153284778/_build_env/bin/x86_64-conda-linux-gnu-nm --ranlib=/croot/ffmpeg_1743153284778/_build_env/bin/x86_64-conda-linux-gnu-ranlib --strip=/croot/ffmpeg_1743153284778/_build_env/bin/x86_64-conda-linux-gnu-strip --disable-doc --enable-swresample --enable-swscale --enable-openssl --enable-libxml2 --enable-libtheora --enable-demuxer=dash --enable-postproc --enable-hardcoded-tables --enable-libfreetype --enable-libharfbuzz --enable-libfont

In [19]:
!ffmpeg -i /data/dataset/HD-EPIC/hd-epic-sounds-trimmed2/P01-20240204-145458-60556944.mp4

ffmpeg version 6.1.1 Copyright (c) 2000-2023 the FFmpeg developers
  built with gcc 11.2.0 (Anaconda gcc)
  configuration: --prefix=/croot/ffmpeg_1743153284778/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_ --cc=/croot/ffmpeg_1743153284778/_build_env/bin/x86_64-conda-linux-gnu-cc --ar=/croot/ffmpeg_1743153284778/_build_env/bin/x86_64-conda-linux-gnu-ar --nm=/croot/ffmpeg_1743153284778/_build_env/bin/x86_64-conda-linux-gnu-nm --ranlib=/croot/ffmpeg_1743153284778/_build_env/bin/x86_64-conda-linux-gnu-ranlib --strip=/croot/ffmpeg_1743153284778/_build_env/bin/x86_64-conda-linux-gnu-strip --disable-doc --enable-swresample --enable-swscale --enable-openssl --enable-libxml2 --enable-libtheora --enable-demuxer=dash --enable-postproc --enable-hardcoded-tables --enable-libfreetype --enable-libharfbuzz --enable-libfont